# Getting Started with QuantumFold-Advantage

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Tommaso-R-Marena/QuantumFold-Advantage/blob/main/examples/01_getting_started.ipynb)

This advanced tutorial introduces **publication-grade** protein structure prediction with quantum machine learning.

## 🚀 Topics Covered
1. **Advanced installation** with ESM-2 and all dependencies
2. **Pre-trained protein embeddings** (ESM-2 from Meta AI)
3. **Quantum-enhanced models** with Invariant Point Attention
4. **Structure prediction** with iterative refinement
5. **Statistical validation** with hypothesis testing
6. **Professional visualization** with confidence scores

## 📚 References
- **ESM-2:** Lin et al., Science (2023) DOI: 10.1126/science.ade2574
- **AlphaFold-3:** Abramson et al., Nature (2024) DOI: 10.1038/s41586-024-07487-w
- **Quantum ML:** Benedetti et al., Quantum Science and Technology (2019)

## 🔧 Step 1: Environment Setup

Install all dependencies. This uses **NumPy 2.x + PennyLane 0.38+** for full Colab compatibility.

In [ ]:
# Check environment
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

import sys
import torch
print(f'🌐 Running in Colab: {IN_COLAB}')
print(f'🔥 PyTorch version: {torch.__version__}')
print(f'⚡ CUDA available: {torch.cuda.is_available()}')

if torch.cuda.is_available():
    print(f'🎮 GPU: {torch.cuda.get_device_name(0)}')
    print(f'💾 Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')
else:
    print('⚠️  No GPU detected. Training will be slower.')
    print('   Enable GPU: Runtime > Change runtime type > T4 GPU')